# wrds_links

> WRDS Concordances

In [ ]:
#| default_exp wrds_links

In [ ]:
#|exports
from __future__ import annotations
from typing import List, Dict, Tuple, Callable
import os

import pandas as pd
import numpy as np
import wrds 

from finsets import wrds_utils

In [ ]:
#| export
def gvkey_permno(wrds_username: str=None,
                 ) -> pd.DataFrame:
    
    sql_string=f"""select gvkey, lpermno as permno, linktype, linkprim, linkdt, linkenddt
                    from crsp.ccmxpf_linktable
                    where substr(linktype,1,1)='L' and (linkprim ='C' or linkprim='P')"""
    
    df = wrds_utils.download(sql_string, wrds_username)
    df['linkdt'] = pd.to_datetime(df['linkdt'])
    df['linkenddt'] = pd.to_datetime(df['linkenddt'])
    df['linkenddt'] = df['linkenddt'].fillna(pd.to_datetime('today'))
    return df

In [ ]:
#| eval: false
#| hide
df = gvkey_permno()
df.head()

Loading library list...
Done


,gvkey,permno,linktype,linkprim,linkdt,linkenddt
0,001000,25881.0,LU,P,1970-11-13,1978-06-30 00:00:00.000000
1,001001,10015.0,LU,P,1983-09-20,1986-07-31 00:00:00.000000
2,001002,10023.0,LC,C,1972-12-14,1973-06-05 00:00:00.000000
3,001003,10031.0,LU,C,1983-12-07,1989-08-16 00:00:00.000000
4,001004,54594.0,LU,P,1972-04-24,2023-06-25 08:30:23.381391


In [ ]:
#| export
def merge_permno_into_gvkey(dset_using_gvkey: pd.DataFrame, # DataFrame using `gvkey` 
          date_var: str='datadate', # Name of date variable in `dset_using_gvkey`
          date_var_format: str="%Y-%m-%d", # Format of `date_var` to be passed to `pd.to_datetime`
          gvkey_permno_link: pd.DataFrame=None, # Concordance DataFrame; will get downloaded if None
          ) -> pd.DataFrame:
    """Merges `permno` into `dset_using_gvkey` using `gvkey_permno_link`."""
    
    if gvkey_permno_link is None: gvkey_permno_link = gvkey_permno()
    df = dset_using_gvkey.merge(gvkey_permno_link, how='left', on=['gvkey'])
    df = df.dropna(subset=[date_var])

    df['dtdate'] = pd.to_datetime(df[date_var], format=date_var_format)    
    df = df[(df['dtdate']>=df['linkdt']) & (df['dtdate']<=df['linkenddt'])].copy()
    df = df.drop(columns = ['linktype', 'linkprim', 'linkdt', 'linkenddt'])
    return df

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()